# Basic Game Loop Tests
Verify Env flow and call resolution using fixed hands.


In [1]:
import os, sys
from pathlib import Path

def find_repo_root(start_dir: str) -> str:
    cur = Path(start_dir).resolve()
    for _ in range(6):
        if (cur / "liars_poker").is_dir() or (cur / "pyproject.toml").exists():
            return str(cur)
        if cur.parent == cur:
            break
        cur = cur.parent
    return str(Path(start_dir).resolve())

NB_DIR = Path.cwd()
REPO_ROOT = Path(find_repo_root(NB_DIR))
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from liars_poker import GameSpec, Env, Rules
from liars_poker.infoset import InfoSet, CALL
from liars_poker.core import card_display, hand_display

print("Repo root:", REPO_ROOT)


Repo root: C:\Users\adidh\Documents\liars_poker


## Scenario A: The Honest Winner
P1 makes a true claim; P2 calls and loses.


In [2]:
spec = GameSpec(ranks=3, suits=1, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=False)
rules = Rules(spec)
env = Env(spec)

# Fixed hands: P1 has rank 2, P2 has rank 1 -> claim RankHigh:2 is true
obs = env.reset(hands=((1,), (0,)))  # card ids since suits=1; ranks map to ids 0-based in legacy encoding
assert env.current_player() == "P1"

claim_true = rules.parse_action("RankHigh:2")
obs = env.step(claim_true)
assert env.current_player() == "P2"

obs = env.step(CALL)
assert obs["terminal"] is True
assert obs["winner"] == "P1", f"Expected P1 to win, got {obs['winner']}"
print("Scenario A passed")


Scenario A passed


## Scenario B: The Caught Bluff
P1 makes a false claim; P2 calls and wins.


In [3]:
spec_b = GameSpec(ranks=3, suits=1, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=False)
rules_b = Rules(spec_b)
env_b = Env(spec_b)

# Fixed hands: P1 rank 1, P2 rank 1 -> claim RankHigh:3 is false
obs = env_b.reset(hands=((0,), (0,)))
claim_false = rules_b.parse_action("RankHigh:3")
obs = env_b.step(claim_false)
obs = env_b.step(CALL)
assert obs["terminal"] is True
assert obs["winner"] == "P2", f"Expected P2 to win, got {obs['winner']}"
print("Scenario B passed")


Scenario B passed


## Scenario C: The Complex Rally
Bid -> Bid -> Bid -> Call with a true Pair claim; caller loses.


In [4]:
spec_c = GameSpec(ranks=4, suits=2, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=False)
rules_c = Rules(spec_c)
env_c = Env(spec_c)

# Fixed hands: ensure P1 has a pair of rank 2, P2 irrelevant for truth

p1_hand = (2, 3)
p2_hand = (0, 1)
obs = env_c.reset(hands=(p1_hand, p2_hand))

# Sequence: P1 bids RankHigh:1, P2 bids RankHigh:2, P1 bids Pair:2 (true), P2 CALLs
bid1 = rules_c.parse_action("RankHigh:1")
bid2 = rules_c.parse_action("RankHigh:2")
pair_claim = rules_c.parse_action("Pair:2")

obs = env_c.step(bid1)   # P1 -> RankHigh:1
obs = env_c.step(bid2)   # P2 -> RankHigh:2
obs = env_c.step(pair_claim)  # P1 -> Pair:2 (true)
obs = env_c.step(CALL)   # P2 calls

assert obs["terminal"] is True
assert obs["winner"] == "P1", f"Expected P1 (claimer) to win, got {obs['winner']}"
print("Scenario C passed")


Scenario C passed


## Scenario D: My Scenario



In [5]:
spec_d = GameSpec(ranks=13, suits=4, hand_size=2, claim_kinds=("RankHigh", "Pair"), suit_symmetry=False)
rules_d = Rules(spec_d)
env_d = Env(spec_d)

# Fixed hands: ensure P1 has a pair of rank 2, P2 irrelevant for truth


p1_hand = (0, 18)
p2_hand = (1, 2)
obs = env_d.reset(hands=(p1_hand, p2_hand))

# Sequence: P1 bids RankHigh:1, P2 bids RankHigh:2, P1 bids Pair:2 (true), P2 CALLs
bid1 = rules_d.parse_action("RankHigh:1")
bid2 = rules_d.parse_action("RankHigh:2")
pair_claim = rules_d.parse_action("Pair:1")

obs = env_d.step(bid1)   # P1 -> RankHigh:1
obs = env_d.step(bid2)   # P2 -> RankHigh:2
obs = env_d.step(pair_claim)  # P1 -> Pair:1 (true)
obs = env_d.step(CALL)   # P2 calls

assert obs["terminal"] is True
assert obs["winner"] == "P1", f"Expected P1 (claimer) to win, got {obs['winner']}"
print("Scenario C passed")


Scenario C passed
